In [1]:
import numpy as np
import scipy as sp
from scipy.special import gammaln
import re, random

In [2]:
def sample_index(p):
    return np.random.multinomial(1,p).argmax()

def word_indices(vec):
    for idx in vec.nonzero()[0]:
        for i in range(int(vec[idx])):
            yield idx

def log_multi_beta(alpha, K=None):
    if K is None:
        return np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))
    else:
        return K * gammaln(alpha) - gammaln(K*alpha)

In [3]:
class LDA:
    def __init__(self, corpus, n_topics, iteration=30, alpha=0.1, beta=0.1):
        self.corpus = corpus
        self.vocabulary = list(set(' '.join(self.corpus).split()))
        self.iteration = iteration
        self.alpha = alpha
        self.beta = beta
        self.n_topics = n_topics
        self._bow()
        n_docs, vocab_size = self.tfidf.shape
        self.nmz = np.zeros((n_docs, n_topics))
        self.nzw = np.zeros((n_topics, vocab_size))
        self.nm = np.zeros(n_docs)
        self.nz = np.zeros(self.n_topics)
        self.topics = {}
        
        for m in range(n_docs):
            for i, w in enumerate(word_indices(self.tfidf[m, :])):
                z = np.random.randint(n_topics)
                self.nmz[m,z] += 1
                self.nm[m] += 1
                self.nzw[z,w] += 1
                self.nz[z] += 1
                self.topics[(m,i)] = z
        
    def _bow(self):
        self.tfidf = np.zeros((len(self.corpus),len(self.vocabulary)))
        for no, i in enumerate(self.corpus):
            for text in i.split():
                self.tfidf[no, self.vocabulary.index(text)] += 1
                
    def _conditional_distribution(self, m, w):
        vocab_size = self.nzw.shape[1]
        left = (self.nzw[:,w] + self.beta) / (self.nz + self.beta * vocab_size)
        right = (self.nmz[m,:] + self.alpha) / (self.nm[m] + self.alpha * self.n_topics)
        p_z = left * right
        p_z /= np.sum(p_z)
        return p_z
                
    def loglikelihood(self):
        vocab_size = self.nzw.shape[1]
        n_docs = self.nmz.shape[0]
        lik = 0
        for z in range(self.n_topics):
            lik += log_multi_beta(self.nzw[z,:]+self.beta)
            lik -= log_multi_beta(self.beta, vocab_size)
        for m in range(n_docs):
            lik += log_multi_beta(self.nmz[m,:]+self.alpha)
            lik -= log_multi_beta(self.alpha, self.n_topics)
        return lik
    
    def run(self):
        for it in range(self.iteration):
            for m in range(self.tfidf.shape[0]):
                for i, w in enumerate(word_indices(self.tfidf[m, :])):
                    z = self.topics[(m,i)]
                    self.nmz[m,z] -= 1
                    self.nm[m] -= 1
                    self.nzw[z,w] -= 1
                    self.nz[z] -= 1
                    p_z = self._conditional_distribution(m, w)
                    z = sample_index(p_z)
                    self.nmz[m,z] += 1
                    self.nm[m] += 1
                    self.nzw[z,w] += 1
                    self.nz[z] += 1
                    self.topics[(m,i)] = z
                

In [4]:
with open('kerajaan','r') as fopen:
    kerajaan = list(filter(None, fopen.read().split('\n')))

In [5]:
def clearstring(string):
    string = re.sub('[^A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    string = ' '.join(string)
    return string.lower()

kerajaan = [clearstring(i) for i in kerajaan]

In [6]:
def show_topics(corpus, count=10, k_words=10):
    lda = LDA(corpus,k_words)
    lda.run()
    vectors = lda.nmz[:count] 
    top_words = lambda t: [lda.vocabulary[i] for i in np.argsort(t)[:-k_words-1:-1]]
    topic_words = ([top_words(t) for t in vectors])
    return [' '.join(t) for t in topic_words]

In [7]:
show_topics(kerajaan)

['buckle airbus bersepadu lawat kemalangn kongres 17rakaat ksemua menampung berliku',
 'kemalangn buckle bersepadu lawat kongres 17rakaat ksemua menampung airbus berliku',
 'kongres bersepadu lawat kemalangn 17rakaat buckle ksemua menampung airbus berliku',
 'kongres bersepadu lawat kemalangn 17rakaat buckle ksemua menampung airbus berliku',
 'menampung bersepadu lawat kemalangn kongres 17rakaat buckle ksemua airbus berliku',
 'kemalangn buckle lawat berliku bersepadu kongres 17rakaat ksemua menampung airbus',
 'menampung airbus bersepadu lawat kemalangn kongres 17rakaat buckle ksemua berliku',
 'kongres bersepadu lawat kemalangn 17rakaat buckle ksemua menampung airbus berliku',
 'ksemua kemalangn buckle airbus bersepadu lawat kongres 17rakaat menampung berliku',
 'berliku buckle kemalangn bersepadu lawat kongres 17rakaat ksemua menampung airbus']